In [1]:
from model.rvae import RVAE
from util.batch_loader import Batch
from util.preprocess import Preprocess
from util.parameter import Parameter
from gensim.models import KeyedVectors
from torch.optim import Adam
import numpy as np
import torch

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### load embedding model

In [2]:
embedding_model=KeyedVectors.load_word2vec_format('embedding.bin')

### load raw data

In [3]:
with open('train.txt','r') as f:
    data=f.readlines()

### preprocess

In [4]:
import os
preprocess=Preprocess(embedding_model)
input=preprocess.to_sequence(data)
if not os.path.exists('embedding.npy'):
    embedding=preprocess.embedding()
    np.save('embedding',embedding)

### batch generator and parameter

In [5]:
batch_loader=Batch(input,0.7)
params=Parameter(word_embed_size=300,encode_rnn_size=300,latent_variable_size=600,\
            decode_rnn_size=300,vocab_size=preprocess.vocab_size,embedding_path='embedding.npy')

### model , optimizer and train

In [6]:
model=RVAE(params)
optimizer=Adam(model.learnable_parameters(), 1e-3)
train_step=model.trainer(optimizer)
if os.path.exists('trained_rvae'):
    model.load_state_dict(torch.load('trained_rvae'))
use_cuda=torch.cuda.is_available()
if use_cuda:
    model=model.cuda()
ce_list=[]
kld_list=[]
coef_list=[]

In [7]:
print(model)

RVAE (
  (encoder): Encoder (
    (highway): Highway (
    )
    (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder (
    (lstm): LSTM(900, 300, batch_first=True)
    (fc): Linear (300 -> 18803)
  )
  (logvar): Linear (600 -> 600)
  (mu): Linear (600 -> 600)
  (embedding): Embedding (
    (word_embed): Embedding(18803, 300)
  )
)


In [8]:
use_cuda

True

### train

In [9]:
test_batch=batch_loader.test_next_batch(1)
for i,batch in enumerate(batch_loader.train_next_batch(1)):

    if i%21==0:
        sample=next(test_batch)
        sentence=model.sample(50,sample,use_cuda)
        sentence=[preprocess.index_to_word[i] for i in sentence]
        print(' '.join(sentence))
        continue
    ce,kld,coef=train_step(i,batch,0.2,use_cuda)
    if i%20==0:
        print('ten step: ce:{}, kld:{} '.format(ce,kld))
        torch.save(model.state_dict(), 'trained_rvae')
    ce_list+=[ce.data.cpu().numpy()[0]]
    kld_list+=[kld.data.cpu().numpy()[0]]
    coef_list+=[coef]
print("epoch finish")

ten step: ce:
 9.6618
[torch.cuda.FloatTensor of size 1 (GPU 0)]
, kld:
 91.0514
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


RuntimeError: cuda runtime error (2) : out of memory at d:\downloads\pytorch-master-1\torch\lib\thc\generic/THCStorage.cu:66

In [2]:
from torch.autograd import Variable


In [19]:
a=Variable(torch.FloatTensor(1))

In [18]:
a=a.cuda()

In [31]:
a.data.cpu().numpy()[0]

array([  2.57838917e-43], dtype=float32)

In [21]:
a.cuda().get_device()

0